In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import Counter
from sklearn.metrics import accuracy_score, log_loss

import pickle
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')
import lightgbm

# read pickle
with open('./data/X.pickle', 'rb') as handle:
    X = pickle.load(handle)
    
with open('./data/y.pickle', 'rb') as handle:
    y = pickle.load(handle)

In [90]:
max_depth_tree =5
num_trees=100

m2 = lightgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=1, max_depth=max_depth_tree,
        min_child_samples=1, min_child_weight=0, min_split_gain=0.0,
        n_estimators=num_trees, n_jobs=-1, num_leaves=31, objective="binary", 
        random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
m2.fit(X, y, verbose=1, eval_set = [(X,y)])

res_2 = m2.predict(X)

accuracy_score(res_2, y)

[1]	training's binary_logloss: 0.364431
[2]	training's binary_logloss: 0.30797
[3]	training's binary_logloss: 0.258656
[4]	training's binary_logloss: 0.239413
[5]	training's binary_logloss: 0.213851
[6]	training's binary_logloss: 0.204589
[7]	training's binary_logloss: 0.199272
[8]	training's binary_logloss: 0.193588
[9]	training's binary_logloss: 0.18745
[10]	training's binary_logloss: 0.172793
[11]	training's binary_logloss: 0.154085
[12]	training's binary_logloss: 0.150774
[13]	training's binary_logloss: 0.130422
[14]	training's binary_logloss: 0.118352
[15]	training's binary_logloss: 0.108654
[16]	training's binary_logloss: 0.09286
[17]	training's binary_logloss: 0.0890692
[18]	training's binary_logloss: 0.0813644
[19]	training's binary_logloss: 0.0745611
[20]	training's binary_logloss: 0.0607006
[21]	training's binary_logloss: 0.0568307
[22]	training's binary_logloss: 0.0529939
[23]	training's binary_logloss: 0.0461759
[24]	training's binary_logloss: 0.0445376
[25]	training's bina

0.9875141884222475

In [91]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X, y, free_raw_data=False)
lgb_eval = lgb.Dataset(X, y, free_raw_data=False)


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
# binary error
def binary_error(preds, train_data):
    labels = train_data.get_label()
    return 'binary_errorrr', np.mean(labels != (preds > 0.5)), False



def accuracy(preds, train_data):
    labels = train_data.get_label()
    return 'accuracy', np.mean(labels == (preds > 0.5)), True



def log_loss_(preds, train_data):
    y_preds = 1. / (1. + np.exp(-preds))
    labels = train_data.get_label()
    return 'log_loss_', log_loss(labels, y_preds), True



# log likelihood loss
def loglikelihood(preds, train_data): #preds here is not y_pred, it's the sum_linear_pred (raw pred)
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess



params = {}
params = {}
params['learning_rate'] = 1
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary'
params['sub_feature'] = 1
params['num_leaves'] = 31
params['min_data'] = 0
params['max_depth'] = max_depth_tree
params['reg_alpha'] = 0
params['reg_lambda'] = 0

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=num_trees,
                fobj=loglikelihood,
                feval=lambda preds, train_data: [log_loss_(preds, train_data),
                                                 accuracy(preds, train_data)],
                valid_sets=lgb_train)

y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)

print(1-np.mean(y != (y_pred > 0.5)))


temp = 1. / (1. + np.exp(-y_pred))
print(1-np.mean(y != (temp > 0.5)))

[1]	training's binary_logloss: 2.6015	training's log_loss_: 0.369754	training's accuracy: 0.830874
[2]	training's binary_logloss: 1.76144	training's log_loss_: 0.316638	training's accuracy: 0.860386
[3]	training's binary_logloss: 2.14285	training's log_loss_: 0.282993	training's accuracy: 0.896708
[4]	training's binary_logloss: 1.26876	training's log_loss_: 0.245081	training's accuracy: 0.911464
[5]	training's binary_logloss: 1.03233	training's log_loss_: 0.228174	training's accuracy: 0.914869
[6]	training's binary_logloss: 0.673541	training's log_loss_: 0.198358	training's accuracy: 0.92622
[7]	training's binary_logloss: 0.371114	training's log_loss_: 0.182951	training's accuracy: 0.934166
[8]	training's binary_logloss: 0.329367	training's log_loss_: 0.212572	training's accuracy: 0.929625
[9]	training's binary_logloss: 0.101863	training's log_loss_: 0.173463	training's accuracy: 0.940976
[10]	training's binary_logloss: -0.256271	training's log_loss_: 0.162633	training's accuracy: 0.94

[94]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[95]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[96]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[97]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[98]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[99]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[100]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
0.7854710556186152
0.7854710556186152


In [93]:
print(accuracy_score(res_2, y))
accuracy_score(res_2, y) == (1-np.mean(y != (y_pred > 0.5)))

0.9875141884222475


False

In [88]:
# lightgbm.create_tree_digraph(m2, tree_index=0, show_info=["split_gain", "internal_value", "internal_count", "leaf_count"] )

In [46]:
# lightgbm.create_tree_digraph(gbm, tree_index=0, show_info=["split_gain", "internal_value", "internal_count", "leaf_count"] )

In [94]:
def loglikelihood(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
# binary error
def binary_error(preds, train_data):
    labels = train_data.get_label()
    return 'error', np.mean(labels != (preds > 0.5)), False


print('Finished 40 - 50 rounds with self-defined objective function and eval metric...')


# another self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
# accuracy
def accuracy(preds, train_data):
    labels = train_data.get_label()
    return 'accuracy', np.mean(labels == (preds > 0.5)), True


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                fobj=loglikelihood,
                feval=lambda preds, train_data: [binary_error(preds, train_data),
                                                 accuracy(preds, train_data)],
                valid_sets=lgb_eval)

Finished 40 - 50 rounds with self-defined objective function and eval metric...
[1]	valid_0's binary_logloss: 2.6015	valid_0's error: 0.169126	valid_0's accuracy: 0.830874
[2]	valid_0's binary_logloss: 1.76144	valid_0's error: 0.139614	valid_0's accuracy: 0.860386
[3]	valid_0's binary_logloss: 2.14285	valid_0's error: 0.103292	valid_0's accuracy: 0.896708
[4]	valid_0's binary_logloss: 1.26876	valid_0's error: 0.0885358	valid_0's accuracy: 0.911464
[5]	valid_0's binary_logloss: 1.03233	valid_0's error: 0.0851305	valid_0's accuracy: 0.914869
[6]	valid_0's binary_logloss: 0.673541	valid_0's error: 0.0737798	valid_0's accuracy: 0.92622
[7]	valid_0's binary_logloss: 0.371114	valid_0's error: 0.0658343	valid_0's accuracy: 0.934166
[8]	valid_0's binary_logloss: 0.329367	valid_0's error: 0.0703746	valid_0's accuracy: 0.929625
[9]	valid_0's binary_logloss: 0.101863	valid_0's error: 0.0590238	valid_0's accuracy: 0.940976
[10]	valid_0's binary_logloss: -0.256271	valid_0's error: 0.0556186	valid_0

In [97]:
y_pred = gbm.predict(X, num_iteration=gbm.best_iteration) > 0.5
accuracy_score(y_pred, y)

0.7854710556186152

objetive = binary   => regresson on (0,1) => 1.1 => sigmoid(1.1) (this is y_pred) => calc error => finish 1 round, return  y_pred=sigmoid(1.1) 

objetive = log_loss_customer_func   => regresson on (0,1) => 1.1 (this is y_pred) => sigmoid(1.1) => calc error => finish 1 round,  return  y_pred=1.1

hessian ?
how to find best split
how to minimize error after know index for best split